In [1]:
from typing import Annotated, Literal,Sequence,TypedDict
from langchain import hub
from langchain_core.messages import AIMessage,HumanMessage,BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import AnyMessage, add_messages
from typing import Any
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda,RunnableWithFallbacks
from langgraph.prebuilt import ToolNode


d:\Demo\langgraph_demo\venv\Lib\site-packages\IPython\core\interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Demo\langgraph_demo\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_groq import ChatGroq

In [5]:
llm=ChatGroq(
    model_name="Gemma2-9b-It")

In [6]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is **Paris**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28, 'completion_time': 0.021818182, 'prompt_time': 0.001905616, 'queue_time': 0.261582444, 'total_time': 0.023723798}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f504b1c2-8be9-4f64-903e-d23a7f9732cb-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28})

In [7]:
urls=[
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"
]

In [8]:
docs = [WebBaseLoader(url).load() for url in urls]

In [9]:
docs

[[Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-11-28-reward-hacking/', 'title': "Reward Hacking in Reinforcement Learning | Lil'Log", 'description': 'Reward hacking occurs when a reinforcement learning (RL) agent exploits flaws or ambiguities in the reward function to achieve high rewards, without genuinely learning or completing the intended task. Reward hacking exists because RL environments are often imperfect, and it is fundamentally challenging to accurately specify a reward function.\nWith the rise of language models generalizing to a broad spectrum of tasks and RLHF becomes a de facto method for alignment training, reward hacking in RL training of language models has become a critical practical challenge. Instances where the model learns to modify unit tests to pass coding tasks, or where responses contain biases that mimic a user’s preference, are pretty concerning and are likely one of the major blockers for real-world deployment of more autonomous use

In [26]:
docs[2][0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/',
 'title': "Prompt Engineering | Lil'Log",
 'description': 'Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics.\nThis post only focuses on prompt engineering for autoregressive language models, so nothing with Cloze tests, image generation or multimodality models. At its core, the goal of prompt engineering is about alignment and model steerability. Check my previous post on controllable text generation.',
 'language': 'en'}

In [27]:
docs[0][0].metadata

{'source': 'https://lilianweng.github.io/posts/2024-11-28-reward-hacking/',
 'title': "Reward Hacking in Reinforcement Learning | Lil'Log",
 'description': 'Reward hacking occurs when a reinforcement learning (RL) agent exploits flaws or ambiguities in the reward function to achieve high rewards, without genuinely learning or completing the intended task. Reward hacking exists because RL environments are often imperfect, and it is fundamentally challenging to accurately specify a reward function.\nWith the rise of language models generalizing to a broad spectrum of tasks and RLHF becomes a de facto method for alignment training, reward hacking in RL training of language models has become a critical practical challenge. Instances where the model learns to modify unit tests to pass coding tasks, or where responses contain biases that mimic a user’s preference, are pretty concerning and are likely one of the major blockers for real-world deployment of more autonomous use cases of AI model

In [28]:
doc_list =[item for sublist in docs for item in sublist]

In [29]:
doc_list

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-11-28-reward-hacking/', 'title': "Reward Hacking in Reinforcement Learning | Lil'Log", 'description': 'Reward hacking occurs when a reinforcement learning (RL) agent exploits flaws or ambiguities in the reward function to achieve high rewards, without genuinely learning or completing the intended task. Reward hacking exists because RL environments are often imperfect, and it is fundamentally challenging to accurately specify a reward function.\nWith the rise of language models generalizing to a broad spectrum of tasks and RLHF becomes a de facto method for alignment training, reward hacking in RL training of language models has become a critical practical challenge. Instances where the model learns to modify unit tests to pass coding tasks, or where responses contain biases that mimic a user’s preference, are pretty concerning and are likely one of the major blockers for real-world deployment of more autonomous use 

In [30]:
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=5)

In [31]:
doc_splits=text_splitter.split_documents(doc_list)

In [ ]:
doc_splits

In [36]:
vector_store=Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome",
    embedding=embeddings
)

In [37]:
retriever=vector_store.as_retriever()

In [10]:
def AI_ASISTANT(
    query: str,
    retriever: Any,
    llm: Any
) -> str:
    """AI Assistant that retrieves relevant documents and generates a response."""
    # Retrieve relevant documents
    docs = retriever.invoke(query)
    
    # Generate response using the LLM
    response = llm.invoke(query, context=docs)
    
    return response

In [11]:
def retrieve(state):
    pass


In [12]:
def rewrite(state):
    pass


In [13]:
def generate(state):
    pass


In [14]:
class AgentState:
    pass




In [15]:
workflow = StateGraph(AgentState)
workflow.add_node("ai_assistant", AI_ASISTANT)
workflow.add_node("retriever", retrieve)
workflow.add_node("rewriter", rewrite) 
workflow.add_node("generate", generate)

In [16]:
workflow.add_edge(START, "retriever")

In [17]:
app =workflow.compile()

In [19]:
app.invoke("hello how are you")

InvalidUpdateError: Expected dict, got hello how are you
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE